This notebook is for creating the base ML model.

In [3]:
from autogluon.timeseries import TimeSeriesPredictor, TimeSeriesDataFrame
import pandas as pd
import numpy as np

In [5]:
df = pd.read_csv('train_2021-22-23-24.csv')
df.drop('Zone', axis=1, inplace=True)

print(df.columns)

Index(['#', 'datetime', 'location', 'busyness'], dtype='object')


In [3]:
df.head()

,#,datetime,location,busyness,Zone
0,1,2021-01-01 00:00:00,3,1,Allerton/Pelham Gardens
1,2,2021-01-01 00:00:00,4,1,Alphabet City
2,3,2021-01-01 00:00:00,5,1,Arden Heights
3,4,2021-01-01 00:00:00,6,1,Arrochar/Fort Wadsworth
4,5,2021-01-01 00:00:00,7,1,Astoria


In [4]:
# Drop the 'zone' column
df = df.drop(columns=['Zone'])

print(df.dtypes)

#            int64
datetime    object
location     int64
busyness     int64
dtype: object


In [9]:
data = TimeSeriesDataFrame.from_data_frame(
    df,
    id_column="location",
    timestamp_column="datetime"
)
print(data.head())

                    #  busyness
item_id timestamp              
3       2021-01-01  1         1
4       2021-01-01  2         1
5       2021-01-01  3         1
6       2021-01-01  4         1
7       2021-01-01  5         1


In [ ]:
prediction_length = 3 * 30 * 24 # 3 months prediction window
train_data, test_data = data.train_test_split(prediction_length)

In [12]:
# setting the frequency to h since the data is grouped/collected hourly,
# # added relative path 

predictor = TimeSeriesPredictor(
    freq='h',
    target="busyness"
    eval_metric="MASE",
    prediction_length=pred_len,
    path="base_model"
)
print('we made the predictor')

we made the predictor


In [18]:
# separating a sample of the train data for testing
subsample_size = 500  # subsample subset of data for faster demo, try setting this to much larger values
test = train_data.sample(n=subsample_size, random_state=0)
test['busyness'] = None
test.tail()

,,#,busyness
item_id,timestamp,,
63,2021-08-22 02:00:00,1436530,None
90,2024-01-16 00:00:00,6850674,None
74,2021-12-22 00:00:00,2188890,None
176,2022-10-27 02:00:00,4096784,None
182,2021-08-30 19:00:00,1490420,None


In [19]:
predictions = predictor.predict(test)
print(predictions)

data with frequency 'None' has been resampled to frequency 'h'.
Model not specified in predict, will default to the model with the best validation score: WeightedEnsemble
Found no cached predictions
Prediction order: ['TemporalFusionTransformer', 'SeasonalNaive', 'WeightedEnsemble']
Predicting with model TemporalFusionTransformer/W0
Predicting with model TemporalFusionTransformer
Shortening all time series to at most 2500
Predicting with model SeasonalNaive/W0
Predicting with model SeasonalNaive
Extending existing cached predictions
Cached predictions saved to autogluon-m4-hourly/models/cached_predictions.pkl


                                 mean       0.1       0.2       0.3       0.4  \
item_id timestamp                                                               
54      2022-07-20 11:00:00  0.024499  0.016303  0.017815  0.019523  0.020714   
        2022-07-20 12:00:00  0.024575  0.016121  0.017457  0.019211  0.020482   
        2022-07-20 13:00:00  0.024608  0.015974  0.017147  0.018884  0.020198   
        2022-07-20 14:00:00  0.024587  0.015869  0.016932  0.018638  0.019950   
        2022-07-20 15:00:00  0.024564  0.015777  0.016758  0.018443  0.019722   
...                               ...       ...       ...       ...       ...   
111     2022-04-13 15:00:00  0.024573  0.015634  0.016650  0.018291  0.019699   
        2022-04-13 16:00:00  0.024597  0.015592  0.016543  0.018142  0.019488   
        2022-04-13 17:00:00  0.024583  0.015589  0.016483  0.018065  0.019333   
        2022-04-13 18:00:00  0.024509  0.015601  0.016434  0.018030  0.019237   
        2022-04-13 19:00:00 

In [12]:
# update the path for github
df1 = pd.read_csv('test_2024_04_03_02.csv')

#making sure there are no duplicates
df1 = df1.drop_duplicates()

# making sure formatting is correct
df1['datetime'] = pd.to_datetime(df1['datetime'])
# Replace all values in the 'busyness' column with NaN
#df1['busyness'] = np.nan
# Replace all values in the 'busyness' column with 100 to test the prediction
df['busyness'] = 100

# making my df a timeseriesdf
dfts= TimeSeriesDataFrame.from_data_frame(
    df1,
    id_column="location",
    timestamp_column="datetime"
)
print(dfts.tail())

                                  #  busyness            Zone
item_id timestamp                                            
265     2024-03-31 23:00:00  370374         2  Outside of NYC
79      2024-04-01 00:00:00  370375         1    East Village
90      2024-04-01 00:00:00  370376         1        Flatiron
107     2024-04-01 00:00:00  370377         1        Gramercy
163     2024-04-01 00:00:00  370378         1   Midtown North


In [8]:
print(predictor.leaderboard())

Generating leaderboard for all models trained


                       model  score_val  pred_time_val  fit_time_marginal  \
0           WeightedEnsemble  -1.419699       3.063145           1.804674   
1  TemporalFusionTransformer  -1.421149       2.549684         182.501629   
2              SeasonalNaive  -2.120057       0.513461           3.800257   
3                        ETS  -5.349771     119.842454           3.847995   
4                      Theta  -7.563938      27.076159           4.023075   
5                      Naive -22.347831       2.718518           3.900895   

   fit_order  
0          6  
1          5  
2          2  
3          3  
4          4  
5          1  


In [13]:

predictions = predictor.predict(dfts)
print(predictions)

data with frequency 'None' has been resampled to frequency 'h'.
Model not specified in predict, will default to the model with the best validation score: WeightedEnsemble
Found no cached predictions
Prediction order: ['TemporalFusionTransformer', 'SeasonalNaive', 'WeightedEnsemble']
Predicting with model TemporalFusionTransformer/W0
Predicting with model TemporalFusionTransformer
Shortening all time series to at most 2500
Predicting with model SeasonalNaive/W0
Predicting with model SeasonalNaive
Extending existing cached predictions
Cached predictions saved to scripts/new_taxi/autogluon_manh_taxi/autogluon-m4-hourly/models/cached_predictions.pkl


                                 mean       0.1       0.2       0.3       0.4  \
item_id timestamp                                                               
1       2024-04-01 00:00:00  1.003467  0.978869  0.998763  0.997993  0.993789   
        2024-04-01 01:00:00  1.003556  0.975604  0.996856  0.996035  0.994512   
        2024-04-01 02:00:00  1.003466  0.975113  0.996011  0.995898  0.994459   
        2024-04-01 03:00:00  1.003206  0.975815  0.995812  0.996920  0.994243   
        2024-04-01 04:00:00  1.002531  0.976436  0.995994  0.998535  0.993697   
...                               ...       ...       ...       ...       ...   
104     2024-03-30 07:00:00  1.000832  0.995501  0.997661  0.998966  0.999819   
        2024-03-30 08:00:00  1.000931  0.995919  0.997807  0.999130  0.999884   
        2024-03-30 09:00:00  1.000924  0.996460  0.998051  0.999213  0.999861   
        2024-03-30 10:00:00  1.000815  0.997082  0.998317  0.999263  0.999815   
        2024-03-30 11:00:00 

In [14]:
plot = predictor.plot(dfts, predictions, quantile_levels=[0.1, 0.9], max_history_length=200, max_num_item_ids=4)
plot.savefig("test-plot.png")

data with frequency 'None' has been resampled to frequency 'h'.
